In [1]:
import os
import time
import pickle
from atproto import Client
import keyring
import shutil
import logging

# Configuration
IDENTIFIER = 'elaval.bsky.social'
RATE_LIMIT_DELAY = 1  # seconds between API calls to respect rate limits

# File paths
GRAPH_FILE = 'user_graph.pkl'
BACKUP_FILE = 'user_graph_backup.pkl'
LOG_FILE = 'update_followers.log'  # Log file for tracking updates

# Default avatar URL (optional, in case you want to handle missing avatars)
DEFAULT_AVATAR = 'https://example.com/default_avatar.png'  # Replace with your default avatar URL

# Configure logging
logging.basicConfig(
    filename=LOG_FILE,  # Log output to a file
    level=logging.INFO,  # Logging level
    format='%(asctime)s - %(levelname)s - %(message)s'  # Log format
)

def get_profile(client, actor):
    """Fetch the profile information for a given actor (handle or DID)."""
    try:
        logging.info(f"Fetching profile for {actor}...")
        profile = client.get_profile(actor=actor)
        logging.info(f"Fetched profile for {actor}.")
        return profile  # Return the profile object
    except Exception as e:
        logging.error(f"Error fetching profile for {actor}: {e}")
        return None

def update_followers_count(client, graph):
    """Update the followers_count and description for each user in the graph."""
    total_users = len(graph)
    updated_users = 0

    for idx, (user_handle, data) in enumerate(graph.items(), start=1):
        logging.info(f"Updating user {idx}/{total_users}: {user_handle}")

        profile = get_profile(client, user_handle)
        if profile:
            # Update followers_count
            new_followers_count = profile.followers_count or 0
            if new_followers_count < 0:
                logging.warning(f"Invalid followers_count for '{user_handle}'. Setting to 0.")
                new_followers_count = 0

            if new_followers_count != data.get('followers_count', 0):
                logging.info(f" - Followers count updated: {data.get('followers_count', 0)} -> {new_followers_count}")
                graph[user_handle]['followers_count'] = new_followers_count
                updated_users += 1
            else:
                logging.info(" - Followers count remains unchanged.")

            # Optionally, update description if needed
            new_description = profile.description or ''
            if new_description != data.get('description', ''):
                logging.info(f" - Description updated for '{user_handle}'.")
                graph[user_handle]['description'] = new_description
        else:
            logging.warning(f" - Failed to fetch profile for '{user_handle}'. Skipping update.")

        time.sleep(RATE_LIMIT_DELAY)  # Respect rate limits

    logging.info(f"Update complete. Total users updated: {updated_users}/{total_users}")
    return graph

def save_graph(graph, graph_file=GRAPH_FILE):
    """Save the updated graph back to the pickle file atomically."""
    temp_file = graph_file + '.tmp'
    try:
        with open(temp_file, 'wb') as f:
            pickle.dump(graph, f)
        shutil.move(temp_file, graph_file)
        logging.info(f"Successfully saved updated graph to '{graph_file}'.")
    except Exception as e:
        logging.error(f"Error saving updated graph: {e}")
        if os.path.exists(temp_file):
            os.remove(temp_file)
        raise e  # Re-raise the exception after cleanup

def main():
    # Retrieve password from keyring
    password = keyring.get_password('bluesky', IDENTIFIER)

    if not password:
        logging.error('No password found in keyring. Please store it first.')
        raise ValueError('No password found in keyring. Please store it first.')

    # Initialize the client and log in
    client = Client()

    try:
        client.login(IDENTIFIER, password)
        logging.info("Login successful.")
    except Exception as e:
        logging.error(f"Login failed: {e}")
        raise e  # Exit the script if login fails

    # Check if the pickle file exists
    if not os.path.exists(GRAPH_FILE):
        logging.error(f"The file '{GRAPH_FILE}' does not exist in the current directory.")
        raise FileNotFoundError(f"The file '{GRAPH_FILE}' does not exist in the current directory.")

    # Backup the original pickle file
    try:
        shutil.copyfile(GRAPH_FILE, BACKUP_FILE)
        logging.info(f"Backup created at '{BACKUP_FILE}'.")
    except Exception as e:
        logging.error(f"Error creating backup: {e}")
        raise e

    # Load the graph data
    with open(GRAPH_FILE, 'rb') as f:
        try:
            graph = pickle.load(f)
            logging.info(f"Successfully loaded '{GRAPH_FILE}'.")
        except Exception as e:
            logging.error(f"Error loading pickle file: {e}")
            raise e

    # Update the followers_count and description
    updated_graph = update_followers_count(client, graph)

    # Save the updated graph
    try:
        save_graph(updated_graph)
    except Exception as e:
        logging.error(f"Failed to save updated graph: {e}")
        raise e

    print(f"Graph updated successfully. Check '{GRAPH_FILE}' and '{LOG_FILE}' for details.")

if __name__ == "__main__":
    main()


Graph updated successfully. Check 'user_graph.pkl' and 'update_followers.log' for details.
